In [2]:
import psycopg2

from sqlalchemy import create_engine
from sqlalchemy import text

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
database_host='10.152.29.90'

In [4]:
def get_connection(host = "localhost",schema="" ):
    
    conn = psycopg2.connect("dbname='proj001_lfb' user='postgres' "
                            "host=" + host )
    if schema:
        cur = conn.cursor()
        query = "SET search_path TO {}".format(schema)
        cur.execute(query)

    return conn

# Create a feature dataset

## Incidents

In [5]:
conn = get_connection(host=database_host)
qry = ("select incidents.*, types.incidentgroup, types.stopcodedescription "
          "from proj001_lfb_1_0_0.l3_incidents incidents "
           "left join "
           "proj001_lfb_1_0_0.l3_incident_types types "
           "on "
           "incidents.id_incident_type = types.id"
          ";")
#data = pd.read_sql(qry, conn).drop(['store_no','store_type'], axis=1)  
df_incidents = pd.read_sql(qry, conn)  
conn.close()

# drop some columns that won't be features
df_incidents.drop(['num_stations_with_pumps_attending', 'num_pumps_attending'], axis =1 ,inplace=True)
df_incidents.head()

,id,incidentnumber_cln,dateofcall_cln,id_incident_type,id_property,id_postcode,incident_station_ground,incidentgroup,stopcodedescription
0,1,79398141,2014-06-22 10:21:15,2,21,108386,OVERTHEBORDER,False Alarm,False alarm - Good intent
1,2,168174131,2013-12-07 04:29:07,6,21,108387,OVERTHEBORDER,Fire,Primary Fire
2,3,167600111,2011-10-14 11:58:03,1,21,108388,BROMLEY,False Alarm,AFA
3,4,107752141,2014-08-12 16:48:29,6,30,108388,BROMLEY,Fire,Primary Fire
4,5,61181121,2012-05-03 19:53:30,1,18,108388,BROMLEY,False Alarm,AFA


## Postcodes

In [17]:
conn = get_connection(host=database_host)
qry = ('select *'
          'from proj001_lfb_1_0_0.l3_postcodes '
          ';')
#data = pd.read_sql(qry, conn).drop(['store_no','store_type'], axis=1)  
df_postcodes = pd.read_sql(qry, conn)  
conn.close()
df_postcodes.head()

,id,postcode_full,postcode_district,incgeo_boroughname,incgeo_wardname,incgeo_wardnamenew,frs
0,1,AL1 3EA,AL1,NOT GEO-CODED,NOT GEO-CODED,NOT GEO-CODED,OVERTHEBORDER
1,2,AL2 1QY,AL2,NOT GEO-CODED,NOT GEO-CODED,NOT GEO-CODED,OVERTHEBORDER
2,3,AL8 7QA,AL8,NOT GEO-CODED,NOT GEO-CODED,NOT GEO-CODED,OVERTHEBORDER
3,4,BR1 1AB,BR1,BROMLEY,BROMLEY TOWN,BROMLEY TOWN,LONDON
4,5,BR1 1AE,BR1,BROMLEY,BROMLEY TOWN,BROMLEY TOWN,LONDON


Here we discover that fields may need to be stripped of trailing spaces in the pipeline code and that a proportion of the postcodes are unlabelled.

In [13]:
df_empty_pcodes = df_postcodes[df_postcodes['incgeo_boroughname']
             .map(lambda x: x.strip()) == 'NOT GEO-CODED'][['postcode_full','incgeo_boroughname']]['postcode_full'].unique()
print("Unlabeled postcodes: " + str(len(pd.DataFrame(df_empty_pcodes)) ))
print("Total postcodes: " + str(len(df_postcodes) ))

Unlabeled postcodes: 1669
Total postcodes: 110157


In [18]:
df_postcodes.drop(['postcode_district','incgeo_wardname','incgeo_wardnamenew'], inplace=True, axis =1 )
df_postcodes.head(2)

,id,postcode_full,incgeo_boroughname,frs
0,1,AL1 3EA,NOT GEO-CODED,OVERTHEBORDER
1,2,AL2 1QY,NOT GEO-CODED,OVERTHEBORDER


## Property details

In [21]:
conn = get_connection(host=database_host)
qry = ('select *'
          'from proj001_lfb_1_0_0.l3_property '
          ';')
#data = pd.read_sql(qry, conn).drop(['store_no','store_type'], axis=1)  
df_property = pd.read_sql(qry, conn)  
conn.close()
df_property.head(3)

,id,propertycategory,propertytype
0,1,Aircraft,Freight plane
1,2,Aircraft,Helicopter
2,3,Aircraft,Light aircraft


# Features dataset

In [22]:
df_incidents.dtypes

id                                  int64
incidentnumber_cln                 object
dateofcall_cln             datetime64[ns]
id_incident_type                    int64
id_property                         int64
id_postcode                         int64
incident_station_ground            object
incidentgroup                      object
stopcodedescription                object
dtype: object